In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
#import mplhep
from matplotlib.colors import LogNorm
from joblib import dump, load

In [ ]:
#proton_selection = "SingleRP"
proton_selection = "MultiRP"

train_model = True
run_grid_search = True
save_model = True

In [ ]:
def get_data( fileNames ):
    df_list = []
    df_counts_list = []

    for file_ in fileNames:
        print ( file_ )
        with h5py.File( file_, 'r' ) as f:
            print ( list(f.keys()) )
            dset = f['protons']
            print ( dset.shape )
            print ( dset[:,:] )

            dset_columns = f['columns']
            print ( dset_columns.shape )
            columns = list( dset_columns )
            print ( columns )
            columns_str = [ item.decode("utf-8") for item in columns ]
            print ( columns_str )

            dset_selections = f['selections']
            selections_ = [ item.decode("utf-8") for item in dset_selections ]
            print ( selections_ )

            dset_counts = f['event_counts']
            df_counts_list.append( pd.Series( dset_counts, index=selections_ ) )
            print ( df_counts_list[-1] )

            chunk_size = 1000000
            entries = dset.shape[0]
            start_ = list( range( 0, entries, chunk_size ) )
            stop_ = start_[1:]
            stop_.append( entries )
            print ( start_ )
            print ( stop_ )
            for idx in range( len( start_ ) ):
                print ( start_[idx], stop_[idx] )
                #print ( dset[ start_[idx] : stop_[idx] ] )
                df_ = pd.DataFrame( dset[ start_[idx] : stop_[idx] ], columns=columns_str )
                df_ = df_[ ['Run', 'LumiSection', 'EventNum', 'CrossingAngle', 
                            'MultiRP', 'Arm', 'RPId1', 'RPId2', 'TrackX1', 'TrackY1', 'TrackX2', 'TrackY2',
                            'Xi', 'T', 'ThX', 'ThY', 'Time',
                            'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuPlus', 'XiMuMuMinus'] ].astype( { "Run": "int64", "LumiSection": "int64", "EventNum": "int64", "MultiRP": "int32", "Arm": "int32", "RPId1": "int32", "RPId2": "int32", "ExtraPfCands": "int32" } )
                df_list.append( df_ )
                print ( df_list[-1].head() )
                print ( len( df_list[-1] ) )

    df_counts = df_counts_list[0]
    for idx in range( 1, len( df_counts_list ) ):
        df_counts = df_counts.add( df_counts_list[idx] )
    print ( df_counts )

    df = pd.concat( df_list )
    print ( df )
    
    return ( df_counts, df )

In [ ]:
def process_data( df ):
    msk = ( df["InvMass"] >= 110. )

    msk1 = None
    msk2 = None
    if proton_selection == "SingleRP":
        # Single-RP in pixel stations
        msk1_arm = ( df["RPId1"] == 23 )
        msk2_arm = ( df["RPId1"] == 123 )
        df[ "XiMuMu" ] = np.nan
        df[ "XiMuMu" ].where( ~msk1_arm, df[ "XiMuMuPlus" ], inplace=True )
        df[ "XiMuMu" ].where( ~msk2_arm, df[ "XiMuMuMinus" ], inplace=True )
        #df_signal[ "XiMuMu" ][ msk2_arm ] = df_signal[ "XiMuMuMinus" ] 
        msk1 = msk & ( df["MultiRP"] == 0) & msk1_arm
        msk2 = msk & ( df["MultiRP"] == 0) & msk2_arm
    elif proton_selection == "MultiRP":
        # Multi-RP
        msk1_arm = ( df["Arm"] == 0 )
        msk2_arm = ( df["Arm"] == 1 )
        df[ "XiMuMu" ] = np.nan
        df[ "XiMuMu" ].where( ~msk1_arm, df[ "XiMuMuPlus" ], inplace=True )
        df[ "XiMuMu" ].where( ~msk2_arm, df[ "XiMuMuMinus" ], inplace=True )
        msk1 = msk & ( df["MultiRP"] == 1 ) & msk1_arm
        msk2 = msk & ( df["MultiRP"] == 1 ) & msk2_arm

    df = df[ msk1 | msk2 ]
    return ( df )    

### Signal

In [ ]:
fileNames_signal = [
    'output/output-MC2017-Elastic-Non3+3-PreSel.h5'
    #'output-MC2017-SingleDissociation-PreSel.h5'
]

df_counts_signal, df_signal = get_data( fileNames_signal )

In [ ]:
df_signal = process_data( df_signal )
df_signal[:20]

### Background

In [ ]:
resample_factor = 20

fileNames_bkg = [
    'output/output-UL2017B-PreSel-Rnd-Res20.h5',
    'output/output-UL2017C1-PreSel-Rnd-Res20.h5',
    'output/output-UL2017E-PreSel-Rnd-Res20.h5',
    'output/output-UL2017F1-PreSel-Rnd-Res20.h5'
]

df_counts_bkg, df_bkg = get_data( fileNames_bkg )

In [ ]:
df_bkg = process_data( df_bkg )
df_bkg[:20]

### Select variables

In [ ]:
X_sig = df_signal[ ['Xi', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ]
print ( X_sig[:20] )

X_bkg = df_bkg[ ['Xi', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ]
print ( X_bkg[:20] )

y_sig = np.ones( len(X_sig) )
y_bkg = np.zeros( len(X_bkg) )

X = pd.concat( [X_sig, X_bkg] ) 
y = np.concatenate( [y_sig, y_bkg] )

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, shuffle=True, random_state=42 )

### Build model (example)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
    
ada_clf = AdaBoostClassifier(
            DecisionTreeClassifier( max_depth=4 ),
            n_estimators = 200,
            algorithm="SAMME.R",
            learning_rate = 0.5)
ada_clf.fit( X_train, y_train )
model = ada_clf

### Evaluate on test data

In [ ]:
y_test_proba = model.predict_proba( X_test )[:,1]
print ( y_test_proba )

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( y_test_proba[ y_test == 0 ], histtype='step', color='orange', bins=60, range=(0.,1.) )
plt.hist( y_test_proba[ y_test == 1 ], histtype='step', color='skyblue', bins=60, range=(0.,1.) )
plt.yscale('log')

### Hyperparameter scan

In [ ]:
grid_search = None

if train_model and run_grid_search:
    from sklearn.model_selection import RandomizedSearchCV
    #from sklearn.model_selection import GridSearchCV
    from scipy.stats import uniform

    param_distribs = {
        "base_estimator__max_depth": np.arange(2,10),
        "n_estimators": 100*np.arange(1,6),
        "learning_rate": uniform()
        }
    #param_grid = [
    #    { "max_depth": np.arange(2,10),
    #      "n_estimators": 100 * np.arange(1,6),
    #      "learning_rate": 0.1 * np.arange(5,11) }
    #    ]

    grid_search = RandomizedSearchCV(
        AdaBoostClassifier(
            DecisionTreeClassifier(),
            algorithm="SAMME.R"
            ),
        param_distribs,
        n_iter=10, cv=3, verbose=20, n_jobs=-1, random_state=42
        )
    grid_search.fit( X_train, y_train )

    print ( grid_search.best_params_ )
    print ( grid_search.best_score_ )
    print ( grid_search.cv_results_ )

In [ ]:
model_final = None

if train_model:
    if run_grid_search: 
        print ( grid_search.best_estimator_)
        model_final = grid_search.best_estimator_.model
    else
        model_final = AdaBoostClassifier(
            DecisionTreeClassifier( max_depth=4 ),
            n_estimators = 200,
            algorithm="SAMME.R",
            learning_rate = 0.5)
        model_final.fit( X_train, y_train )
else:
    model_final = load( "model/ada_clf.joblib" )
    
print ( model_final )

### Evaluate on test data

In [ ]:
y_test_proba = model_final.predict_proba( X_test )[:,1]
print ( y_test_proba )

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( y_test_proba[ y_test == 0 ], histtype='step', color='orange', bins=60, range=(0.,1.) )
plt.hist( y_test_proba[ y_test == 1 ], histtype='step', color='skyblue', bins=60, range=(0.,1.) )
plt.yscale('log')

In [ ]:
prob_cut = 0.50

y_test_pred = ( y_test_proba >= prob_cut ).astype( "int32" )
print ( y_test_pred )

from sklearn.metrics import accuracy_score
print ( accuracy_score( y_test, y_test_pred ) )
print ( accuracy_score( y_test[ y_test == 1 ], y_test_pred[ y_test == 1 ] ) )
print ( accuracy_score( y_test[ y_test == 0 ], y_test_pred[ y_test == 0 ] ) )

### Save model

In [ ]:
if train_model and save_model:
    dump( model_final, "model/ada_clf.joblib" )

### References

In [ ]:
from scipy.stats import uniform
np.info(uniform)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
np.info( AdaBoostClassifier )

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#np.info( RandomizedSearchCV )
print ( RandomizedSearchCV(AdaBoostClassifier(
            DecisionTreeClassifier( random_state=42 ),
            algorithm="SAMME.R"
            ),
            param_distribs)
      )